# Steganografija slike

Način na koji se može sakriti tajna poruka tokom slanja tako da ne privlači pažnju napadača jeste steganografija. U kriptografiji poruke su šifrovane te često zainteresuju napadače koji pokušavaju da razbiju šifru, dok u steganografiji sakrivena poruka se ne zaključava vidljivim ključem koji se potom odvojeno šalje, pa ne postoji objekat privlačenja pogleda napadača.
<hr>
<i>Steganografija</i> je metoda sakrivanja tajne poruke u slici, zvuku, videu itd koji će biti prenosni mediji. Kako tekstualna poruka, tako se i slika može sakriti u drugoj slici i slično. Takvom steganografijom ćemo se baviti u nastavku.

Tehnika sakrivanja poruke/slike u slici-nosaču jeste ubacivanje značajnih bitova (bitova velike težine) tajne poruke/slike na mestima najnižih bitova slike-nosača, jer na taj način slika-nosač neće biti vidljivo promenjena i tajna poruka/slika neće biti vidljiva. Bitno je izabrati dobru sliku-nosač koja je puna detalja, jer male promene u pikselima koji predstavljaju oštre ivice mogu biti vidljive. Dakle, ono što će biti promenjeno jeste intenzitet RGB boja u nekim pikselima. Procenat originalne slike koji možemo menjati je mali, 10-ak procenata da promena slike ne bi bila vidljiva. 

<hr>
Prikazivanje svakog piksela je moguće čuvanjem vrednosti boja u njemu. Koristićemo RGB model boja. Svaki piksel će imati tri 8-bitne vrednosti za svaku od boja.

![rgb](rgb.png)
<hr>
<img src="tajno.png" width="500">

Kako bismo dobili male promene slike-nosača menjaćemo vrednosti njenih najnižih bitova sa bitovima najveće težine poruke ili slike koju želimo da sakrijemo. 

Svaki piksel će biti predstavljen trojkama 8-bitnih brojeva, svaki će predstavljati jačinu jedne od RGB boja. Vrednosti idu od 0 do 255.
![rgb_bit](rgb_bit.png)

Za početak sakrićemo sliku unutar druge slike:

In [2]:
%config IPCompleter.greedy=True

In [71]:
from PIL import Image
import numpy as np

def copy_image(image):
    return Image.new(image.mode, image.size, (0,0,0))

def get_size(image):
    return (image.size[0], image.size[1])


class Steganography:
    #From 3 given numbers this returns rgb values in binary representation
    get_binary_rgb = lambda r, g, b: (format(r, "b").zfill(8), format(g, "b").zfill(8), (format(b, "b").zfill(8)))
  
    @staticmethod
    def get_rgb(image_rgb):
         return Steganography.get_binary_rgb(image_rgb[0], image_rgb[1], image_rgb[2])

    
    #Method that returns new image that is a result of merging secret image 
    #into holder image that will be send and visible
    @staticmethod
    def encrypt(image_holder, image_secret):
        #If image size (with and hight given as a tuple) is bigger then holder image, than we want to exit
        h_weight, h_hight = get_size(image_holder)
        s_weight, s_hight = get_size(image_secret)
        if s_weight > h_weight or s_weight > h_hight:
            raise ValueError('Secret image is bigger than holder image!')
        
        #We can see RGB values for all pixcels with command: #print np.asarray(image_holder)
        
        #Pixel maps for both images
        image_holder_pixels = image_holder.load()
        image_secret_pixels = image_secret.load()
       
        #Output image will be the same as holder, and then we are going to change it's pixels
        #importing part of pixels of secret image
        output_image = copy_image(image_holder)
        output_image_pixels = output_image.load()
        
        for i in range(h_weight):
            for j in range(h_hight): 
                rgb_binary_holder = Steganography.get_rgb(image_holder_pixels[i,j])
                 
                #Check if the secret image is finished because it is smaller than holder image         
                if(s_weight > i and s_hight > j):
                    rgb_binary_secret = Steganography.get_rgb(image_secret_pixels[i,j])
                else:
                    rgb_binary_secret = Steganography.get_rgb((0, 0, 0))
                
                #Now we have rgb binary values for holder and secret image
                #Then we want to place secret image into holder one into last 4 bits
                r = rgb_binary_holder[0][:4] + rgb_binary_secret[0][:4]
                g = rgb_binary_holder[1][:4] + rgb_binary_secret[1][:4]
                b = rgb_binary_holder[2][:4] + rgb_binary_secret[2][:4]
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2))
                
        return output_image
        
    
    #Method that returns secret image that is hidden in visible recived image
    @staticmethod
    def decrypt(image):
        #Pixel map for image
        image_pixels = image.load()
      
        weight, hight = get_size(image)
        size = image.size
        
        #Output image will be the same as holder, and then we are going to change it's pixels
        #importing part of pixels of secret image
        output_image = Image.new(image.mode, size, (0, 0, 0))
        output_image_pixels = output_image.load()
        
        #For size of secret image
        k = 0
        l = 0
        
        #Making rgb map and checking if all pixels are zero then there is no hidden picture
        for i in range(weight):
            for j in range(hight): 
                rgb_binary = Steganography.get_rgb(image_pixels[i,j])
                
                #Now we have rgb binary values
                #Then we want to get secret image from holder one by taking last 4 bits
                r = rgb_binary[0][4:] + '0000'
                g = rgb_binary[1][4:] + '0000'
                b = rgb_binary[2][4:] + '0000'
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2))
                
                #If all pixels are zero, then there is no secret picture merged on that pixel
                #We are seting size of secret picture, last pixel not zero is end of the secret image
                #so we are saving that pixel indexes
                if output_image_pixels[i, j] == (0, 0, 0):
                    pass
                else:
                    k = i
                    l = j
        
        output_weight = k + 1
        output_hight = l + 1
        #Crop picture from first pixel (left upper corner) to size that we found
        output_image = output_image.crop((0, 0, output_weight, output_hight))        
            
        return output_image
        
        
        
        
        #image_holder.close()
        #image_secret.close()
        
#if __name__ == '__main__':
#encrypt = Steganography.encrypt(Image.open('New_york.jpg'), Image.open('secret_photo.jpg'))       
#encrypt.save('output.png')
decrypt = Steganography.decrypt(Image.open('output.png'))       
decrypt.save('output_dec.png')